In [2]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import torch
from torch.utils.data import TensorDataset, DataLoader
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [5]:
with open("X_test.pkl", 'rb') as f:
    X_test = pickle.load(f)

with open("X_train.pkl", 'rb') as f:
    X_train = pickle.load(f)

with open("y_test.pkl", 'rb') as f:
    y_test = pickle.load(f)

with open("y_train.pkl", 'rb') as f:
    y_train = pickle.load(f)
    
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 4 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
import torch.nn as nn
import torch.nn.functional as F

# PyTorch models inherit from torch.nn.Module
class HeartDiseaseClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HeartDiseaseClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.output = nn.Linear(64, output_dim)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        #x = torch.sigmoid(self.output(x))
        
        return x


model = HeartDiseaseClassifier(input_dim = X_train.shape[1] , output_dim = 1).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)


In [7]:
def calculate_accuracy(outputs, labels):
    # Apply sigmoid to logits and threshold at 0.5 for binary classification
    preds = torch.sigmoid(outputs) >= 0.5  # Converts logits to binary predictions
    return (preds == labels).float().mean()  # Calculate mean accuracy


In [8]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    running_acc = 0.
    last_loss = 0.
    last_acc = 0.
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        
        optimizer.zero_grad()
        
        outputs = model(inputs).squeeze()  # Squeeze to match the label shape
        loss = loss_fn(outputs, labels.squeeze())  # Match dimensions
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        batch_accuracy = calculate_accuracy(outputs, labels)
        running_acc += batch_accuracy.item()
        if i % 16 == 15:
            last_loss = running_loss / 16 # loss per batch
            last_acc = running_acc / 16  # Average accuracy over the last 16 batches
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            tb_writer.add_scalar('Accuracy/train', last_acc, tb_x)
            running_loss = 0.
            running_acc = 0.
            
    return last_loss

In [12]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/heart_disease_classifier{}'.format(timestamp))
epoch_number = 0

EPOCHS = 1000

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    

    running_vloss = 0.0
    model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)  # Move data to GPU
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss
    
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
    
    epoch_number += 1

EPOCH 1:
  batch 16 loss: 0.7076363936066628
  batch 32 loss: 0.693256001919508
  batch 48 loss: 0.6946496963500977
LOSS train 0.6946496963500977 valid 0.7017516493797302
EPOCH 2:
  batch 16 loss: 0.7009462937712669
  batch 32 loss: 0.6964547075331211
  batch 48 loss: 0.6957279145717621
LOSS train 0.6957279145717621 valid 0.698940634727478
EPOCH 3:
  batch 16 loss: 0.6866101436316967
  batch 32 loss: 0.6905080303549767
  batch 48 loss: 0.6943470761179924
LOSS train 0.6943470761179924 valid 0.696040689945221
EPOCH 4:
  batch 16 loss: 0.6865958534181118
  batch 32 loss: 0.6890987306833267
  batch 48 loss: 0.6987191289663315
LOSS train 0.6987191289663315 valid 0.693253755569458
EPOCH 5:
  batch 16 loss: 0.6868382133543491
  batch 32 loss: 0.6888525262475014
  batch 48 loss: 0.68861248716712
LOSS train 0.68861248716712 valid 0.6902115941047668
EPOCH 6:
  batch 16 loss: 0.6910265646874905
  batch 32 loss: 0.6877324432134628
  batch 48 loss: 0.6826207004487514
LOSS train 0.6826207004487514 v

In [ ]:
# Define new code for training and testing with parameter sweeps.
from itertools import product
import torch.optim as optim

# Parameters to sweep
batch_sizes = [4, 16, 32]
learning_rates = [0.001, 0.01, 0.1]
optimizers = ['SGD', 'Adam', 'RMSprop']

# Placeholder for results
results = []

# Updated training loop with parameter sweeps
for batch_size, lr, opt_name in product(batch_sizes, learning_rates, optimizers):
    print(f"testing with batch size {batch_size}, lr {lr}, optimiser {opt_name}")
    # Reinitialize dataloaders with current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Reinitialize the model
    model = HeartDiseaseClassifier(input_dim=X_train.shape[1], output_dim=1).to(device)
    
    # Configure optimizer
    if opt_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif opt_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif opt_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    
    # Loss function
    loss_fn = nn.BCEWithLogitsLoss()
    
    # Training process
    num_epochs = 750
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
    # Evaluation on test set
    model.eval()
    test_accuracy = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            test_accuracy += calculate_accuracy(outputs, y_batch).item()
    test_accuracy /= len(test_loader)
    
    # Save results
    results.append({
        'Batch Size': batch_size,
        'Learning Rate': lr,
        'Optimizer': opt_name,
        'Test Accuracy': test_accuracy
    })

# Convert results to a DataFrame for better readability
import pandas as pd
results_df = pd.DataFrame(results)
results_df.sort_values(by='Test Accuracy', ascending=False, inplace=True)
results_df


KeyboardInterrupt: 